<a href="https://colab.research.google.com/github/leonardoub/cmepda/blob/master/BRATS_anova_fselect_nested_CV_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

#Load data

In [5]:
#load data from Drive
from google.colab import drive
drive.mount('/gdrive')
#%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
dataset_path = '/gdrive/My Drive/BRATS/data_without_NAN_without_HISTO_without_SPATIAL_with_histologies.csv'

In [0]:
df_data = pd.read_csv(dataset_path)

In [8]:
df_data

,ID,Date,VOLUME_ET,VOLUME_NET,VOLUME_ED,VOLUME_TC,VOLUME_WT,VOLUME_BRAIN,VOLUME_ET_OVER_NET,VOLUME_ET_OVER_ED,VOLUME_NET_OVER_ED,VOLUME_ET_over_TC,VOLUME_NET_over_TC,VOLUME_ED_over_TC,VOLUME_ET_OVER_WT,VOLUME_NET_OVER_WT,VOLUME_ED_OVER_WT,VOLUME_TC_OVER_WT,VOLUME_ET_OVER_BRAIN,VOLUME_NET_OVER_BRAIN,VOLUME_ED_over_BRAIN,VOLUME_TC_over_BRAIN,VOLUME_WT_OVER_BRAIN,DIST_Vent_TC,DIST_Vent_ED,INTENSITY_Mean_ET_T1Gd,INTENSITY_STD_ET_T1Gd,INTENSITY_Mean_ET_T1,INTENSITY_STD_ET_T1,INTENSITY_Mean_ET_T2,INTENSITY_STD_ET_T2,INTENSITY_Mean_ET_FLAIR,INTENSITY_STD_ET_FLAIR,INTENSITY_Mean_NET_T1Gd,INTENSITY_STD_NET_T1Gd,INTENSITY_Mean_NET_T1,INTENSITY_STD_NET_T1,INTENSITY_Mean_NET_T2,INTENSITY_STD_NET_T2,INTENSITY_Mean_NET_FLAIR,...,TEXTURE_NGTDM_ED_T1_Busyness,TEXTURE_NGTDM_ED_T1_Complexity,TEXTURE_NGTDM_ED_T1_Strength,TEXTURE_NGTDM_ED_T2_Coarseness,TEXTURE_NGTDM_ED_T2_Contrast,TEXTURE_NGTDM_ED_T2_Busyness,TEXTURE_NGTDM_ED_T2_Complexity,TEXTURE_NGTDM_ED_T2_Strength,TEXTURE_NGTDM_ED_FLAIR_Coarseness,TEXTURE_NGTDM_ED_FLAIR_Contrast,TEXTURE_NGTDM_ED_FLAIR_Busyness,TEXTURE_NGTDM_ED_FLAIR_Complexity,TEXTURE_NGTDM_ED_FLAIR_Strength,TEXTURE_NGTDM_NET_T1Gd_Coarseness,TEXTURE_NGTDM_NET_T1Gd_Contrast,TEXTURE_NGTDM_NET_T1Gd_Busyness,TEXTURE_NGTDM_NET_T1Gd_Complexity,TEXTURE_NGTDM_NET_T1Gd_Strength,TEXTURE_NGTDM_NET_T1_Coarseness,TEXTURE_NGTDM_NET_T1_Contrast,TEXTURE_NGTDM_NET_T1_Busyness,TEXTURE_NGTDM_NET_T1_Complexity,TEXTURE_NGTDM_NET_T1_Strength,TEXTURE_NGTDM_NET_T2_Coarseness,TEXTURE_NGTDM_NET_T2_Contrast,TEXTURE_NGTDM_NET_T2_Busyness,TEXTURE_NGTDM_NET_T2_Complexity,TEXTURE_NGTDM_NET_T2_Strength,TEXTURE_NGTDM_NET_FLAIR_Coarseness,TEXTURE_NGTDM_NET_FLAIR_Contrast,TEXTURE_NGTDM_NET_FLAIR_Busyness,TEXTURE_NGTDM_NET_FLAIR_Complexity,TEXTURE_NGTDM_NET_FLAIR_Strength,TGM_p1,TGM_dw,TGM_Cog_X_1,TGM_Cog_Y_1,TGM_Cog_Z_1,TGM_T_1,Histology
0,TCGA-02-0006,1996.08.23,1662,384,36268,2046,38314,1469432,4.328125,0.045826,0.010588,0.812320,0.187680,17.726300,0.043378,0.010022,0.946599,0.053401,0.001131,0.000261,0.024682,0.001392,0.026074,31.5903,2.7735,149.7977,10.4671,194.1422,15.1037,154.9225,43.4709,220.5894,30.2917,137.8881,6.3820,183.6933,14.8846,161.1005,35.8591,227.7510,...,0.86315,1479.9762,1.10870,0.000605,0.40937,1.47070,2992.2698,0.71642,0.000690,0.28977,1.8815,1872.0528,0.75986,0.026040,0.37869,0.060929,1675.0041,14.11380,0.044156,0.41942,0.026740,2536.7559,43.31290,0.036634,0.50304,0.024264,3593.3279,43.67590,0.057204,0.33980,0.021897,2203.2034,61.32930,8.00000,7.500000e-07,0.178609,0.096256,0.052741,2.00000,GBM
1,TCGA-02-0009,1997.06.14,4362,4349,15723,8711,24434,1295721,1.002989,0.277428,0.276601,0.500750,0.499250,1.805000,0.178522,0.177990,0.643489,0.356511,0.003366,0.003356,0.012135,0.006723,0.018857,9.2443,3.0207,165.4345,6.4047,201.2400,13.4733,113.1601,10.1373,210.1810,15.9543,152.6013,4.2360,188.0607,11.1316,116.8538,10.0992,209.7901,...,0.40004,2378.9184,2.54730,0.000914,0.70926,0.78063,5719.2847,1.29980,0.000882,0.48919,1.8243,2954.8148,0.77199,0.002254,0.29324,1.223600,539.3057,0.53125,0.005712,0.20995,0.315580,967.7845,3.74440,0.003790,0.36163,0.271420,1996.1440,2.77050,0.004966,0.28715,0.189980,1440.4285,3.59990,3.31250,1.000000e-09,0.077618,0.122900,0.094336,91.47360,GBM
2,TCGA-02-0011,1998.02.01,33404,48612,45798,82016,127814,1425843,0.687155,0.729377,1.061444,0.407290,0.592710,0.558400,0.261349,0.380334,0.358318,0.641682,0.023428,0.034094,0.032120,0.057521,0.089641,1.0000,1.0000,186.3385,17.6126,188.2019,23.5195,172.8969,32.7401,167.1395,34.1684,149.0643,12.9090,158.4197,15.2632,197.4966,27.1781,165.1014,...,1.51780,1750.3404,0.56482,0.000382,0.59301,1.81810,4990.3388,0.54747,0.000345,0.59184,2.4243,4703.9458,0.41937,0.000403,0.37863,1.957500,2509.3979,0.42842,0.000768,0.19849,1.395800,1322.6082,0.74730,0.000634,0.31856,1.144300,2517.8629,0.84294,0.000794,0.17961,1.068800,1147.5177,0.80480,5.78125,1.000000e-09,0.132283,0.116006,0.096035,272.42900,GBM
3,TCGA-02-0027,1999.03.28,12114,7587,34086,19701,53787,1403429,1.596679,0.355395,0.222584,

In [9]:
df_data.columns

Index(['ID', 'Date', 'VOLUME_ET', 'VOLUME_NET', 'VOLUME_ED', 'VOLUME_TC',
       'VOLUME_WT', 'VOLUME_BRAIN', 'VOLUME_ET_OVER_NET', 'VOLUME_ET_OVER_ED',
       ...
       'TEXTURE_NGTDM_NET_FLAIR_Busyness',
       'TEXTURE_NGTDM_NET_FLAIR_Complexity',
       'TEXTURE_NGTDM_NET_FLAIR_Strength', 'TGM_p1', 'TGM_dw', 'TGM_Cog_X_1',
       'TGM_Cog_Y_1', 'TGM_Cog_Z_1', 'TGM_T_1', 'Histology'],
      dtype='object', length=578)

In [0]:
data = df_data.drop(['Histology', 'ID', 'Date'], axis=1)

In [0]:
labels = df_data.Histology

In [0]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
labels_encoded = encoder.fit_transform(labels)

In [28]:
encoder.classes_

array(['GBM', 'LGG'], dtype=object)

#ANOTHER WAY

In [0]:
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,KFold,cross_val_predict,cross_val_score,StratifiedKFold

from sklearn.svm import SVC
from collections import Counter
from sklearn.metrics import classification_report,accuracy_score

from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_classif

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler


In [0]:
# Create an SelectKBest object to select features with two best ANOVA F-Values
fvalue_selector = SelectKBest(f_classif, k=8)
fvalue_selector_percentile = SelectPercentile(f_classif, percentile=10)

In [0]:
# Designate distributions to sample hyperparameters from 
C = [0.001, 0.01, 0.1, 1, 10]
n_features_to_test = [0.9]
depth = [10, None]

In [0]:
# Choose cross-validation techniques for the inner and outer loops,
# independently of the dataset.
# E.g "GroupKFold", "LeaveOneOut", "LeaveOneGroupOut", etc.
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
outer_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

In [0]:
#RandomForestClassifier
steps = [('scaler', StandardScaler()), ('red_dim', SelectPercentile(f_classif, percentile=10)), ('clf', SVC(random_state=503))]
pipeline = Pipeline(steps)

parameteres = [{'scaler':[StandardScaler()], 
                'clf__C':list(C)}]

In [0]:
from sklearn.metrics import roc_auc_score

In [52]:
outer_loop_roc_auc_scores = []
outer_loop_accuracy_scores = []
inner_loop_won_params = []
inner_loop_roc_auc_scores = []
inner_loop_accuracy_scores = []
best_est_dict = {}
#best_feat_dict = {}
i = 0

# Looping through the outer loop, feeding each training set into a GSCV as the inner loop
for train_index, test_index in outer_cv.split(data, labels_encoded):
    
    i+=1

    GSCV = GridSearchCV(pipeline, param_grid=parameteres, cv=inner_cv, n_jobs=-1, scoring=['roc_auc', 'accuracy'], refit='roc_auc', verbose=1)
    
    # GSCV is looping through the training data to find the best parameters. This is the inner loop
    GSCV.fit(data.iloc[train_index, :], labels_encoded[train_index])
    
    # The best hyper parameters from GSCV is now being tested on the unseen outer loop test data.
    pred = GSCV.predict(data.iloc[test_index, :])
    

    #per far uscire i best_estimators in qualche modo
    best_est_dict.update({f'best_est_{i}' : GSCV.best_estimator_})

    #The most important 10 features for the best estimator
    #features_list =  sorted(zip(rank_features, data.columns), reverse=True)[:10]
    #best_feat_dict.update({f'best_set_HP_{i}' : })


    # Appending the "winning" hyper parameters and their associated accuracy score
    inner_loop_won_params.append(GSCV.best_params_)
    
    outer_loop_roc_auc_scores.append(roc_auc_score(labels_encoded[test_index], pred))
    outer_loop_accuracy_scores.append(accuracy_score(labels_encoded[test_index], pred))

    inner_loop_roc_auc_scores.append(GSCV.best_score_)
    #inner_loop_accuracy_scores.append(GSCV.best_score_)

for i in zip(inner_loop_won_params, outer_loop_roc_auc_scores, inner_loop_roc_auc_scores):
    print(i)

#print('Mean of outer loop accuracy score:' np.mean(outer_loop_roc_auc_scores))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 3 folds for each of 5 candidates, totalling 15 fits
({'clf__C': 1, 'scaler': StandardScaler(copy=True, with_mean=True, with_std=True)}, 0.818627450980392, 0.949202166593471)
({'clf__C': 0.01, 'scaler': StandardScaler(copy=True, with_mean=True, with_std=True)}, 0.5, 0.9103718342848778)
({'clf__C': 0.001, 'scaler': StandardScaler(copy=True, with_mean=True, with_std=True)}, 0.5, 0.9355731225296443)


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.2s finished


In [53]:
inner_loop_roc_auc_scores

[0.949202166593471, 0.9103718342848778, 0.9355731225296443]

In [54]:
outer_loop_roc_auc_scores

[0.818627450980392, 0.5, 0.5]

In [0]:
p_values = GSCV.best_estimator_.named_steps['red_dim'].pvalues_

In [75]:
p_values

array([1.46474982e-07, 5.72466570e-07, 9.82757734e-01, 1.12593307e-01,
       3.14486203e-01, 5.22093330e-02, 8.14512027e-03, 2.06441279e-05,
       2.32666436e-04, 2.50636058e-12, 2.50626757e-12, 2.87860480e-02,
       5.29338937e-11, 9.79933379e-09, 3.79841823e-01, 3.79841823e-01,
       1.23330775e-07, 3.32696106e-07, 8.41935844e-01, 1.65421595e-01,
       4.71131105e-01, 1.59495582e-01, 1.57470550e-03, 1.03504994e-03,
       3.71949071e-03, 2.24897531e-02, 1.32976814e-01, 7.96202173e-01,
       9.24542477e-07, 4.32274304e-01, 2.76324903e-04, 1.19491087e-02,
       3.23958302e-01, 2.93106927e-03, 7.67506810e-01, 2.00068688e-01,
       1.07550441e-03, 8.98522006e-01, 7.06063626e-01, 8.40305415e-01,
       4.34975098e-04, 3.51270716e-02, 3.54696270e-02, 4.11501108e-01,
       1.92275205e-02, 7.83780244e-01, 3.79922649e-01, 6.97951022e-01,
       1.64830030e-01, 5.51804552e-01, 1.34858442e-02, 5.99152492e-01,
       1.69943526e-06, 1.02422279e-01, 4.38339362e-01, 3.14887725e-02,
      

In [0]:
mask_percentile = GSCV.best_estimator_.named_steps['red_dim'].get_support()

In [0]:
features = data.columns

In [72]:
features.shape

(575,)

In [0]:
dict_features_pvalue = {'features':features[mask_percentile], 'p_value':p_values[mask_percentile]}

In [65]:
dict_features_pvalue

{'features': Index(['VOLUME_ET', 'VOLUME_NET', 'VOLUME_ET_over_TC', 'VOLUME_NET_over_TC',
        'VOLUME_ET_OVER_WT', 'VOLUME_NET_OVER_WT', 'VOLUME_ET_OVER_BRAIN',
        'VOLUME_NET_OVER_BRAIN', 'INTENSITY_STD_ET_T2', 'SOLIDITY_NET',
        'TEXTURE_GLOBAL_ET_T2_Variance', 'TEXTURE_GLOBAL_ET_T2_Kurtosis',
        'TEXTURE_GLCM_ET_T1Gd_Homogeneity', 'TEXTURE_GLCM_ET_T2_SumAverage',
        'TEXTURE_GLCM_ET_T2_Variance', 'TEXTURE_GLCM_ET_T2_AutoCorrelation',
        'TEXTURE_GLCM_ET_FLAIR_SumAverage', 'TEXTURE_GLCM_ET_FLAIR_Variance',
        'TEXTURE_GLCM_ET_FLAIR_AutoCorrelation', 'TEXTURE_GLRLM_ET_T1Gd_SRE',
        'TEXTURE_GLRLM_ET_T1Gd_LRE', 'TEXTURE_GLRLM_ET_T1Gd_RLN',
        'TEXTURE_GLRLM_ET_T1Gd_RP', 'TEXTURE_GLRLM_ET_T1Gd_GLV',
        'TEXTURE_GLRLM_ET_T1_SRE', 'TEXTURE_GLRLM_ET_T1_LRE',
        'TEXTURE_GLRLM_ET_T1_RLN', 'TEXTURE_GLRLM_ET_T1_RP',
        'TEXTURE_GLRLM_ET_T1_GLV', 'TEXTURE_GLRLM_ET_T1_RLV',
        'TEXTURE_GLRLM_ET_T2_GLN', 'TEXTURE_GLRLM_ET_T2_HGRE',


In [0]:
df_features_pvalue = pd.DataFrame.from_dict(dict_features_pvalue)

In [67]:
df_features_pvalue

,features,p_value
0,VOLUME_ET,1.464750e-07
1,VOLUME_NET,5.724666e-07
2,VOLUME_ET_over_TC,2.506361e-12
3,VOLUME_NET_over_TC,2.506268e-12
4,VOLUME_ET_OVER_WT,5.293389e-11
5,VOLUME_NET_OVER_WT,9.799334e-09
6,VOLUME_ET_OVER_BRAIN,1.233308e-07
7,VOLUME_NET_OVER_BRAIN,3.326961e-07
8,INTENSITY_STD_ET_T2,9.245425e-07
9,SOLIDITY_NET,1.699435e-06


In [0]:
df_features_pvalue_sorted = df_features_pvalue.sort_values(by=['p_value'], ascending=False)

In [74]:
df_features_pvalue_sorted

,features,p_value
57,TEXTURE_NGTDM_NET_FLAIR_Busyness,7.423370e-06
50,TEXTURE_GLSZM_ET_FLAIR_ZSV,6.849178e-06
12,TEXTURE_GLCM_ET_T1Gd_Homogeneity,5.456383e-06
54,TEXTURE_NGTDM_ET_T2_Strength,5.263705e-06
51,TEXTURE_NGTDM_ET_T1Gd_Busyness,3.904349e-06
29,TEXTURE_GLRLM_ET_T1_RLV,3.443463e-06
10,TEXTURE_GLOBAL_ET_T2_Variance,2.517909e-06
34,TEXTURE_GLRLM_ET_T2_GLV,2.322110e-06
52,TEXTURE_NGTDM_ET_T1Gd_Strength,2.252219e-06
48,TEXTURE_GLSZM_ET_FLAIR_GLN,2.164151e-06


#VEDIAMO LE FEATURES IN MANIERA ORGANICA PER TUTTI I K FOLD OUTER

In [0]:
features = data.columns

In [79]:
best_est_dict

{'best_est_1': Pipeline(memory=None,
          steps=[('scaler',
                  StandardScaler(copy=True, with_mean=True, with_std=True)),
                 ('red_dim',
                  SelectPercentile(percentile=10,
                                   score_func=<function f_classif at 0x7f65da177bf8>)),
                 ('clf',
                  SVC(C=1, break_ties=False, cache_size=200, class_weight=None,
                      coef0=0.0, decision_function_shape='ovr', degree=3,
                      gamma='scale', kernel='rbf', max_iter=-1,
                      probability=False, random_state=503, shrinking=True,
                      tol=0.001, verbose=False))],
          verbose=False), 'best_est_2': Pipeline(memory=None,
          steps=[('scaler',
                  StandardScaler(copy=True, with_mean=True, with_std=True)),
                 ('red_dim',
                  SelectPercentile(percentile=10,
                                   score_func=<function f_classif at 0x7f65d

In [0]:
best_est_1 = best_est_dict['best_est_1']
best_est_2 = best_est_dict['best_est_2']
best_est_3 = best_est_dict['best_est_3']

In [0]:
p_values_1 = best_est_1.named_steps['red_dim'].pvalues_
mask_percentile_1 = best_est_1.named_steps['red_dim'].get_support()
dict_features_pvalue_1 = {'features':features[mask_percentile_1], 'p_value':p_values_1[mask_percentile_1]}
df_features_pvalue_1 = pd.DataFrame.from_dict(dict_features_pvalue_1)
df_features_pvalue_sorted_1 = df_features_pvalue_1.sort_values(by=['p_value'], ascending=False)

In [82]:
df_features_pvalue_sorted_1

,features,p_value
20,TEXTURE_GLRLM_ET_T1_LRLGE,2.036350e-06
29,TEXTURE_GLRLM_ET_FLAIR_SRE,2.023176e-06
45,TEXTURE_GLSZM_ET_T1_ZSV,1.984307e-06
51,TEXTURE_GLSZM_ET_FLAIR_SZLGE,1.756778e-06
31,TEXTURE_GLRLM_ET_FLAIR_RLN,1.692402e-06
40,TEXTURE_GLSZM_ET_T1_SZE,9.151246e-07
25,TEXTURE_GLRLM_ET_T2_SRHGE,8.578774e-07
16,TEXTURE_GLRLM_ET_T1_LRE,8.300146e-07
53,TEXTURE_NGTDM_ET_T1Gd_Coarseness,6.323449e-07
36,TEXTURE_GLSZM_ET_T1Gd_SZE,6.104959e-07


In [85]:
for key, value in best_est_dict.items():
  p_values_1 = value.named_steps['red_dim'].pvalues_
  mask_percentile_1 = value.named_steps['red_dim'].get_support()
  dict_features_pvalue_1 = {'features':features[mask_percentile_1], 'p_value':p_values_1[mask_percentile_1]}
  df_features_pvalue_1 = pd.DataFrame.from_dict(dict_features_pvalue_1)
  df_features_pvalue_sorted_1 = df_features_pvalue_1.sort_values(by=['p_value'], ascending=True)
  print(df_features_pvalue_sorted_1)

                              features       p_value
1                    VOLUME_ET_over_TC  4.852018e-14
2                   VOLUME_NET_over_TC  4.852117e-14
14           TEXTURE_GLRLM_ET_T1Gd_GLV  3.530432e-12
34          TEXTURE_GLRLM_ET_FLAIR_GLV  2.020245e-11
26           TEXTURE_GLRLM_ET_T2_LRHGE  1.623376e-10
10  TEXTURE_GLCM_ET_T2_AutoCorrelation  1.986310e-10
35          TEXTURE_GLRLM_ET_FLAIR_RLV  2.809563e-10
8        TEXTURE_GLCM_ET_T2_SumAverage  4.038087e-10
4                   VOLUME_NET_OVER_WT  4.431485e-10
21             TEXTURE_GLRLM_ET_T1_GLV  7.573270e-10
3                    VOLUME_ET_OVER_WT  1.740971e-09
27             TEXTURE_GLRLM_ET_T2_GLV  2.275058e-09
28             TEXTURE_GLRLM_ET_T2_RLV  3.980447e-09
42              TEXTURE_GLSZM_ET_T1_ZP  5.886966e-09
9          TEXTURE_GLCM_ET_T2_Variance  1.086130e-08
56   TEXTURE_NGTDM_ET_FLAIR_Coarseness  2.105741e-08
38            TEXTURE_GLSZM_ET_T1Gd_ZP  3.288716e-08
19           TEXTURE_GLRLM_ET_T1_SRLGE  3.3820